In [1]:
import pandas as pd

Data = pd.read_csv("unigram_freq.csv")

In [2]:
import string
import re
import numpy as np

In [3]:
# build dictionaries
WORD_FREQ   = dict(zip(Data["word"], Data["count"]))
TOTAL_FREQ  = sum(WORD_FREQ.values())
WORD_PROB   = {w: c / TOTAL_FREQ for w, c in WORD_FREQ.items()}

VOCAB       = set(WORD_FREQ) 

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer

In [5]:
def delete_letter(word):
    return [word[:i] + word[i+1:] for i in range(len(word))]

In [6]:
def swap_letters(word):
    return [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1)]


In [7]:
def replace_letter(word):
    letters = string.ascii_lowercase
    return [word[:i] + c + word[i+1:] for i in range(len(word)) for c in letters]


In [8]:
def insert_letter(word):
    letters = string.ascii_lowercase
    return [word[:i] + l + word[i:] for i in range(len(word)+1) for l in letters]

In [9]:
from collections import Counter

from tqdm.auto import tqdm
import  itertools, math


In [10]:
!pip install rapidfuzz.string_metric

ERROR: Could not find a version that satisfies the requirement rapidfuzz.string_metric (from versions: none)
ERROR: No matching distribution found for rapidfuzz.string_metric


In [11]:
def generate(word):
    return set(
        delete_letter(word)
        + swap_letters(word)
        + replace_letter(word)
        + insert_letter(word)
    )


In [12]:
def known(words, vocab):
    return [w for w in words if w in vocab]


In [13]:
def correct_word(word, vocab=VOCAB, prob=WORD_PROB):
    word = word.lower()
    if word in vocab:
        return word
    
    candidates = known(generate(word), vocab)
    
    if not candidates:
        return word  # no correction found
    
    return max(candidates, key=lambda w: prob.get(w, 1e-9))  # most probable


In [14]:
def correct_sentence(sentence):
    words = sentence.split()
    corrected = [correct_word(w.strip(string.punctuation)) for w in words]
    return ' '.join(corrected)



In [15]:
print(correct_sentence("thiss is an exmple of txt with errrors"))


this is an example of txt with errors
